In [100]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install transformers
!pip install torch
!pip install transformers datasets evaluate
!pip install madgrad

  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached tokenizers-0.13.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached xxhash-3.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (213 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 15.5 MB/s eta 0:00:0000:0100:01
  Using cached multiprocess-0.70.14-py37-none-any.whl (

In [7]:
import torch
import transformers
#import datasets
import json
import pandas as pd
import numpy as np
#import evaluate
#from datasets import Dataset
import torch.utils.data as data_utils
import torch.utils.data
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification
from transformers import AutoImageProcessor, ViTMAEModel, VisionTextDualEncoderProcessor, VisionTextDualEncoderModel
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from PIL import Image
import torch.nn as nn
import tqdm
import torch.optim as optim
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoImageProcessor,
    AutoTokenizer,ViTMAEConfig
)
from madgrad import MADGRAD
import tensorflow as tf

2023-04-28 23:43:20.629059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 23:43:25.838702: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-28 23:43:34.906179: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-28 23:43:34.906631: W tensorflow/strea

In [8]:
train = pd.read_json("data/train.jsonl",lines=True)
test = pd.read_json("data/test.jsonl",lines=True)
val = pd.read_json("data/dev.jsonl",lines=True)

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda:0'

# Embeddings

In [11]:
go = False

In [12]:
if go:
    tokenizer_name = "jayanta/ProsusAI-finbert-sentiment-finetuned-memes"
    processor_name = "google/vit-base-patch16-224"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    image_processor = AutoImageProcessor.from_pretrained(processor_name)

    dual_processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)
    encoder_model = VisionTextDualEncoderModel.from_vision_text_pretrained(processor_name, tokenizer_name)

In [13]:
if go:
    for i in range(0,len(train),400):
        if i == 0:
            encoder_model = VisionTextDualEncoderModel.from_vision_text_pretrained(processor_name, tokenizer_name)
        else:
            encoder_model = VisionTextDualEncoderModel.from_pretrained("encoder_model")
        trains = train.loc[i:i+399,:]
        images = [Image.open('data/'+path).convert("RGB") for path in trains['img']]
        inputs = dual_processor(text=list(trains['text']), images=images, return_tensors="pt", padding=True)
        outputs = encoder_model(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        pixel_values=inputs.pixel_values,
        return_loss=True,
                                )
        #to_save = {dict((k, outputs[k]) for k in ('image_embeds', 'text_embeds','text_model_output','vision_model_output'))}
        to_save = {'image_embeds':outputs['image_embeds'],'text_embeds':outputs['text_embeds'],'text_model_output':outputs['text_model_output']['last_hidden_state'],'vision_model_output':outputs['vision_model_output']['last_hidden_state']}
        name = f'output_checkpoints/outputs_{i}'
        torch.save(to_save,name)
        encoder_model.save_pretrained('encoder_model')
        torch.cuda.empty_cache()

In [14]:
# Now do the same for validation

In [15]:
if go:
    for i in range(0,len(val),250):
        if i == 0:
            encoder_model = VisionTextDualEncoderModel.from_vision_text_pretrained(processor_name, tokenizer_name)
        else:
            encoder_model = VisionTextDualEncoderModel.from_pretrained("encoder_model2_throwaway")
        trains = val.loc[i:i+249,:]
        images = [Image.open('data/'+path).convert("RGB") for path in trains['img']]
        inputs = dual_processor(text=list(trains['text']), images=images, return_tensors="pt", padding=True)
        outputs = encoder_model(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        pixel_values=inputs.pixel_values,
        return_loss=True,
                                )
        #to_save = {dict((k, outputs[k]) for k in ('image_embeds', 'text_embeds','text_model_output','vision_model_output'))}
        to_save = {'image_embeds':outputs['image_embeds'],'text_embeds':outputs['text_embeds'],'text_model_output':outputs['text_model_output']['last_hidden_state'],'vision_model_output':outputs['vision_model_output']['last_hidden_state']}
        name = f'output_validation/outputs_{i}'
        torch.save(to_save,name)
        encoder_model.save_pretrained('encoder_model2_throwaway')
        torch.cuda.empty_cache()

# Loading the Data Back: 

In [53]:
for i in range(0,len(train),20):
    
    t = f'output_checkpoints/outputs_{i}'
    temp = torch.load(t)
    im_emb = temp['image_embeds'][0:20,:].unsqueeze(1)
    txt_emb = temp['text_embeds'][0:20,:].unsqueeze(1)
    final_emb_t = torch.concat([im_emb,txt_emb],axis=1)
    
    text_hidden = torch.zeros(temp['vision_model_output'][0:20,:,:].shape)
    source = temp['text_model_output'][0:20,:,:]
    text_hidden[:, 0:temp['text_model_output'].shape[1], :] = source
    text_hidden = text_hidden.unsqueeze(1)
    vision_hidden = temp['vision_model_output'][0:20,:,:].unsqueeze(1)
    
    final_hidden_t = torch.concat([vision_hidden,text_hidden],axis=1)
    
    if i==0:
        final_embeds = final_emb_t
        final_hidden = final_hidden_t
    else:
        final_embeds = torch.concat([final_embeds,final_emb_t],axis=0)
        final_hidden = torch.concat([final_hidden,final_hidden_t],axis=0)
        

In [54]:
for i in range(0,len(val),250):
    
    t = f'output_validation/outputs_{i}'
    temp = torch.load(t)
    im_emb = temp['image_embeds'][0:250,:].unsqueeze(1)
    txt_emb = temp['text_embeds'][0:250,:].unsqueeze(1)
    final_emb_t = torch.concat([im_emb,txt_emb],axis=1)
    
    text_hidden = torch.zeros(temp['vision_model_output'][0:250,:,:].shape)
    source = temp['text_model_output'][0:250,:,:]
    text_hidden[:, 0:temp['text_model_output'].shape[1], :] = source
    text_hidden = text_hidden.unsqueeze(1)
    vision_hidden = temp['vision_model_output'][0:250,:,:].unsqueeze(1)
    
    final_hidden_t = torch.concat([vision_hidden,text_hidden],axis=1)
    
    if i==0:
        final_embeds_val = final_emb_t
        final_hidden_val = final_hidden_t
    else:
        final_embeds_val = torch.concat([final_embeds_val,final_emb_t],axis=0)
        final_hidden_val = torch.concat([final_hidden_val,final_hidden_t],axis=0)
        

In [63]:
final_embeds_grad = final_embeds.data.numpy().reshape((final_embeds.shape[0],final_embeds.shape[1]*final_embeds.shape[2]))
final_hidden_grad = final_hidden.data.numpy().reshape((final_hidden.shape[0],final_hidden.shape[1]*final_hidden.shape[2]*final_hidden.shape[3]))
final_embeds_val_flat = final_embeds_val.data.numpy().reshape((final_embeds_val.shape[0],final_embeds_val.shape[1]*final_embeds_val.shape[2]))
final_hidden_val_flat = final_hidden_val.data.numpy().reshape((final_hidden_val.shape[0],final_hidden_val.shape[1]*final_hidden_val.shape[2]*final_hidden_val.shape[3]))

# Final Model

In [101]:
gc.collect()
torch.cuda.empty_cache()


tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1, noise_shape=None),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.0005),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='AUC')
    ]
)
history = model.fit(final_hidden_grad, np.array(list(train.loc[:,'label'])), epochs=50)

Epoch 1/50
266/266 [==============================] - 7s 22ms/step - loss: 1.1074 - accuracy: 0.5852 - precision: 0.4124 - recall: 0.3672 - AUC: 0.5522
Epoch 2/50
266/266 [==============================] - 7s 25ms/step - loss: 0.7171 - accuracy: 0.6424 - precision: 0.5020 - recall: 0.4193 - AUC: 0.6414
Epoch 3/50
266/266 [==============================] - 10s 39ms/step - loss: 0.6413 - accuracy: 0.6775 - precision: 0.5628 - recall: 0.4538 - AUC: 0.6951
Epoch 4/50
266/266 [==============================] - 10s 39ms/step - loss: 0.5643 - accuracy: 0.7274 - precision: 0.6465 - recall: 0.5302 - AUC: 0.7644
Epoch 5/50
266/266 [==============================] - 8s 30ms/step - loss: 0.5099 - accuracy: 0.7667 - precision: 0.7015 - recall: 0.6089 - AUC: 0.8210
Epoch 6/50
266/266 [==============================] - 6s 22ms/step - loss: 0.4280 - accuracy: 0.8045 - precision: 0.7540 - recall: 0.6754 - AUC: 0.8740
Epoch 7/50
266/266 [==============================] - 6s 22ms/step - loss: 0.3744 - ac

In [103]:
results = model.evaluate(final_hidden_val_flat, np.array(val['label']), batch_size=500)

1/1 [==============================] - 2s 2s/step - loss: 3.0448 - accuracy: 0.5300 - precision: 0.5510 - recall: 0.3240 - AUC: 0.5380
